In [ ]:
#import libraries
import pandas as pd
import numpy as np
from datetime import datetime, date
import warnings
warnings.simplefilter("ignore")

In [ ]:
team_report_filename = "TeamReport" 
user_report_filename = "UserReport" 
team_task_report_filename = "TeamTasks" 

In [ ]:
#read data
team_report_closed_df = pd.read_excel("../CDM Reports/" + team_report_filename + ".xlsx",
                                      usecols=["Task ID",
                                               "Task Type",
                                               "Process Type",
                                               "Case No",
                                               "Task Closure Date",
                                               "Currrent Task Status",
                                               "Task Assigned to Date",
                                               "Team",
                                               "Team Defined SLA (in days)",
                                               "Time taken (in days) aggregated",
                                               "Generic Volume"])
user_report_df = pd.read_excel("../CDM Reports/" + user_report_filename + ".xlsx",
                               usecols=["Case No",
                                        "Task ID",
                                        "Process Type",
                                        "Currrent Task Status",
                                        "User",
                                        "Task Assigned to Date",
                                        "Team",
                                        "Actual Time Taken"])
team_task_report_df = pd.read_excel("../CDM Reports/" + team_task_report_filename + ".xlsx",
                                    usecols=["Case No"])
dashboard_df = pd.read_excel("Data.xlsx", sheet_name="Base")
bc_list_df = pd.read_excel("Data.xlsx",sheet_name="Bulk Creation")
gv_list_df = pd.read_excel("Data.xlsx",sheet_name="Generic Volume")

***

In [ ]:
#remove rows with no case number and no process type and no task ID
team_report_closed_df = team_report_closed_df.dropna(axis=0, subset=["Task ID", "Case No", "Process Type"])
user_report_df = user_report_df.dropna(axis=0, subset=["Task ID", "Case No", "Process Type"])

In [ ]:
#select rows with process type == "SG - ..."
team_report_closed_df = team_report_closed_df[team_report_closed_df["Process Type"].str.contains("SG - ")]
user_report_df = user_report_df[user_report_df["Process Type"].str.contains("SG - ")]

In [ ]:
#select rows for completed/rejected tasks only
team_report_closed_df = team_report_closed_df[(team_report_closed_df["Currrent Task Status"]=="COMPLETED") |
                                              (team_report_closed_df["Currrent Task Status"]=="REJECTED")]
user_report_df = user_report_df[(user_report_df["Currrent Task Status"]=="COMPLETED") |
                                (user_report_df["Currrent Task Status"]=="REJECTED")]

In [ ]:
#strip all trailing whitespace:
team_report_closed_df = team_report_closed_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
user_report_df = user_report_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
team_task_report_df = team_task_report_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
dashboard_df = dashboard_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
bc_list_df = bc_list_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
gv_list_df = gv_list_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
#remove cases that also appear in the Team Task Report (cases that are still open)
team_report_closed_df = team_report_closed_df[~team_report_closed_df["Case No"].isin(team_task_report_df["Case No"].tolist())]

In [ ]:
#convert date cells to datetime object type
team_report_closed_df["Task Closure Date"] = pd.to_datetime(team_report_closed_df["Task Closure Date"])
team_report_closed_df["Task Assigned to Date"] = pd.to_datetime(team_report_closed_df["Task Assigned to Date"])
user_report_df["Task Assigned to Date"] = pd.to_datetime(user_report_df["Task Assigned to Date"])

In [ ]:
#list of teams and processes used for dashboard
process_list = dashboard_df["Process Type"].tolist()
team_list = dashboard_df["Team"].tolist()

In [ ]:
#exceptions for bulk creation and generic volume
bc_list = bc_list_df.to_numpy().tolist()
gv_list = gv_list_df.to_numpy().tolist()

***

In [ ]:
#sort ascending according to Case No, Task ID, Task Assigned to Date, Time taken (in days) aggregated
team_report_closed_df = team_report_closed_df.sort_values(["Case No", 
                                                           "Task ID", 
                                                           "Task Assigned to Date", 
                                                           "Time taken (in days) aggregated"])
user_report_df = user_report_df.sort_values(["Case No", 
                                             "Task ID", 
                                             "Task Assigned to Date", 
                                             "Actual Time Taken"])

In [ ]:
#organize into a list of cases, and list of tasks ordered by the respective cases
case_list = pd.unique(team_report_closed_df["Case No"]).tolist()
task_list_temp = []
for case in case_list:
    temp_df = team_report_closed_df[team_report_closed_df["Case No"]==case]
    task_list_temp.append(pd.unique(temp_df["Task ID"]).tolist())
task_list = [item for sublist in task_list_temp for item in sublist]

In [ ]:
#recalculating the number of days taken - excluding weekends. to be in line with CDM logic, public holidays will not be excluded
new_time_taken_temp = []
calendar_days_temp = []
for task in task_list:
    #get list of all dates in "Task Assigned to Date" for ONE task, sorted
    temp_df = team_report_closed_df[team_report_closed_df["Task ID"]==task]
    #create a list of the dates (in chronological order) where tasks are assigned to teams
    assigned_dates = pd.to_datetime(temp_df["Task Assigned to Date"]).tolist()
    #add task closure date to end of list
    closure_date = list(set(pd.to_datetime(temp_df["Task Closure Date"]).tolist()))
    assigned_dates.append(closure_date[0])
    
    #find difference between adjacent dates in the list (1. diff by business days, 2. diff by calendar days)
    new_time_taken_temp.append([np.busday_count(j.date(), i.date()) for i,j in zip(assigned_dates[1:], assigned_dates[:-1])])
    calendar_days_temp.append([(i.date() - j.date()).days for i,j in zip(assigned_dates[1:], assigned_dates[:-1])])

In [ ]:
#create new column for new re-calculated time taken that excludes weekends and phs
#flatten the temp lists so i can convert them into columns
new_time_taken = [item for sublist in new_time_taken_temp for item in sublist]
calendar_days = [item for sublist in calendar_days_temp for item in sublist]

In [ ]:
team_report_closed_df["New Time Taken"] = new_time_taken
team_report_closed_df["Calendar Days"] = calendar_days

***

In [ ]:
#this is where- we cross-reference with the user report
#flag out the tasks where calendar days ! = time taken aggregate (means that tasks were reassigned out at some point)
assigned_out_df = team_report_closed_df[team_report_closed_df["Time taken (in days) aggregated"] != team_report_closed_df["Calendar Days"]]

In [ ]:
#compare to user report to get the accurate time taken
#get list of all the flagged tasks
assigned_out_task_list = pd.unique(assigned_out_df["Task ID"]).tolist()
#create a new list for storage
new_user_time_taken_temp = []

#loop through tasks in the flagged task list 
for task in assigned_out_task_list:
    temp_df = team_report_closed_df[team_report_closed_df["Task ID"]==task]
    temp_user_df = user_report_df[user_report_df["Task ID"]==task]

    #this should create a list of all the dates where the tasks were reassigned + closure date (in chronological order)
    assigned_user_dates = pd.to_datetime(temp_user_df["Task Assigned to Date"]).tolist()
    closure_date = list(set(pd.to_datetime(temp_df["Task Closure Date"]).tolist()))
    assigned_user_dates.append(closure_date[0])
    
    #find difference between adjacent dates in the list
    new_user_time_taken_temp.append([np.busday_count(j.date(), i.date()) for i,j in zip(assigned_user_dates[1:], assigned_user_dates[:-1])])

In [ ]:
new_user_time_taken = [item for sublist in new_user_time_taken_temp for item in sublist] #this list is too long. where are the extra values coming from
#match back to user report since calculations were made with user report data
user_report_df = user_report_df[user_report_df["Task ID"].isin(assigned_out_task_list)]
user_report_df["Time Taken (User Report)"] = new_user_time_taken


In [ ]:
#get a list of the total time (.sum()) taken per task per team, based on the user report. store in time_list
time_list = []
for task, team in zip(assigned_out_df["Task ID"],assigned_out_df["Team"]):
    #time taken, in business days, based on user report
    x = user_report_df[(user_report_df["Task ID"]==task) & (user_report_df["Team"]==team)]["Time Taken (User Report)"].sum()
    time_list.append(x)

In [ ]:
#"New Time Taken" for flagged tasks are immediately replaced with the Time Taken based on the User Report
assigned_out_df["New Time Taken"] = time_list

In [ ]:
#merge team_report_closed_df and assigned_out_df back together
#1. remove the rows from team_report_closed_df that correspond to assigned_out_df
#2. append/concat the full assigned_out_df dataframe to team_report_closed_df

to_drop = assigned_out_df.index.values.tolist()
team_report_closed_df = team_report_closed_df.drop(to_drop)
team_report_closed_df = pd.concat([team_report_closed_df, assigned_out_df])

In [ ]:
#this is the part where we take the minimum of the new time taken and the time taken aggregate to account for tasks that were assigned out to Front/IT/etc.
team_report_closed_df["Min Time Taken"] = team_report_closed_df[["New Time Taken","Time taken (in days) aggregated"]].min(axis=1)

***

In [ ]:
#bulk creation: creating dataframe
bulk_creation_df = pd.DataFrame()
for idx in range(len(bc_list)):    
    temp_df = team_report_closed_df[(team_report_closed_df["Team"]==bc_list[idx][0]) &
                                    (team_report_closed_df["Process Type"]==bc_list[idx][1]) &
                                    (team_report_closed_df["Task Type"]==bc_list[idx][2])]
    bulk_creation_df = pd.concat([bulk_creation_df,temp_df])

In [ ]:
#bulk creation: counting volume
bc_completed_data = []
bc_completed_list = []
bc_completed_cases = []
bc_rejected_data = []
bc_rejected_list = []
bc_rejected_cases = []
for idx in range(len(process_list)):
    
    #completed bulk creation tasks
    bc_completed_df = bulk_creation_df[(bulk_creation_df["Process Type"]==process_list[idx]) &
                                       (bulk_creation_df["Currrent Task Status"]=="COMPLETED") &
                                       (bulk_creation_df["Team"]==team_list[idx])]
    
    #sort by case number - this is not for the script, this is for me to visualise better
    bc_completed_df = bc_completed_df.sort_values(by=["Case No", "Task Type"])

    #for each case: volume=max([len(tasks) for each task type]) - for each case: sum # of tasks per task type, then take the maximum
    #temp_list = []
    temp_dict = {}
    for case in list(pd.unique(bc_completed_df["Case No"])): #for each case, get list of unique task types
        unique_tasktype_list = list(pd.unique(bc_completed_df[bc_completed_df["Case No"]==case]["Task Type"]))
        
        bc_volume_by_task = []
        for tasktype in unique_tasktype_list: #for each task type within case, calculate bulk creation volume: (len(unique(task IDs)))
            bc_volume_by_task.append(len(pd.unique(bc_completed_df[(bc_completed_df["Case No"]==case) & (bc_completed_df["Task Type"]==tasktype)]["Task ID"])))
        
        bc_volume_by_case = max(bc_volume_by_task) #bc volume for each case is max(volume of task type 1, volume of task type 2) )
        #temp_list.append([case, bc_volume_by_case]) #append to temp_list [(case1, volume), (case2, volume), (case3, volume), etc.]
        temp_dict[case] = bc_volume_by_case
        
    bc_completed_data.append(temp_dict) #append to bc_completed_data [temp_list for Process1TeamA, temp_list for Process2TeamA, etc.]
    bc_completed_list.append(list(pd.unique(bc_completed_df[bc_completed_df["Team Defined SLA (in days)"] < bc_completed_df["Min Time Taken"]]["Case No"])))
    bc_completed_cases.append(list(pd.unique(bc_completed_df["Case No"])))                                                             
    
    #rejected bulk creation tasks
    bc_rejected_df = bulk_creation_df[(bulk_creation_df["Process Type"]==process_list[idx]) &
                                      (bulk_creation_df["Currrent Task Status"]=="REJECTED") &
                                      (bulk_creation_df["Team"]==team_list[idx])]
    bc_rejected_df = bc_rejected_df[~bc_rejected_df["Task ID"].isin(bc_completed_df["Task ID"].tolist())] #remove tasks already counted under "COMPLETED" so we won't double count
    
    #sort by case number - this is not for the script, this is for me to visualise better
    bc_rejected_df = bc_rejected_df.sort_values(by=["Case No", "Task Type"])

    #for each case: volume=max([len(tasks) for each task type]) - for each case: sum # of tasks per task type, then take the maximum
    #temp_list = []
    temp_dict = {}
    for case in list(pd.unique(bc_rejected_df["Case No"])): #for each case, get list of unique task types
        unique_tasktype_list = list(pd.unique(bc_rejected_df[bc_rejected_df["Case No"]==case]["Task Type"]))
        
        bc_volume_by_task = []
        for tasktype in unique_tasktype_list: #for each task type within case, calculate bulk creation volume: (len(unique(task IDs)))
            bc_volume_by_task.append(len(pd.unique(bc_rejected_df[(bc_rejected_df["Case No"]==case) & (bc_rejected_df["Task Type"]==tasktype)]["Task ID"])))

        bc_volume_by_case = max(bc_volume_by_task) #bc volume for each case is max(volume of task type 1, volume of task type 2) )
        #temp_list.append([case, bc_volume_by_case]) #append to temp_list [(case1, volume), (case2, volume), (case3, volume), etc.]
        temp_dict[case] = bc_volume_by_case
            
    bc_rejected_data.append(temp_dict) #append to bc_completed_data [temp_list for Process1TeamA, temp_list for Process2TeamA, etc.]
    bc_rejected_list.append(list(pd.unique(bc_rejected_df[bc_rejected_df["Team Defined SLA (in days)"] < bc_rejected_df["Min Time Taken"]]["Case No"])))
    bc_rejected_cases.append(list(pd.unique(bc_rejected_df["Case No"])))

***

In [ ]:
#generic volume: creating dataframe
generic_volume_df = pd.DataFrame()
for idx in range(len(gv_list)):
    temp_df = team_report_closed_df[(team_report_closed_df["Team"]==gv_list[idx][0]) &
                                    (team_report_closed_df["Process Type"]==gv_list[idx][1]) &
                                    (team_report_closed_df["Task Type"]==gv_list[idx][2])]
    generic_volume_df = pd.concat([generic_volume_df,temp_df])

In [ ]:
#generic volume: preprocessing. fill all blank cells, convert each cell to list and take the last element of the list
generic_volume_df["Generic Volume"] = generic_volume_df["Generic Volume"].fillna('1')
generic_volume_df["Generic Volume"] = generic_volume_df["Generic Volume"].astype(str) #necessary step in the case where all generic volume entries are miraculously somehow all integers
generic_volume_df["Generic Volume"] = generic_volume_df["Generic Volume"].apply(lambda x: list(x.split(","))[-1]) #last entry of generic volume is the latest input in CDM task
generic_volume_df["Generic Volume"] = (generic_volume_df["Generic Volume"].astype(float)).astype(int)

In [ ]:
#generic volume: counting volume
gv_completed_data = []
gv_completed_list = []
gv_completed_cases = []
gv_rejected_data = []
gv_rejected_list = []
gv_rejected_cases = []
for idx in range(len(process_list)):
    
    #completed generic volume tasks
    gv_completed_df = generic_volume_df[(generic_volume_df["Process Type"]==process_list[idx]) &
                                        (generic_volume_df["Currrent Task Status"]=="COMPLETED") &
                                        (generic_volume_df["Team"]==team_list[idx])] 
    
    #sort by case number - this is not for the script, this is for me to visualise better
    gv_completed_df = gv_completed_df.sort_values(by=["Case No", "Task Type"])
    
    #temp_list = []
    temp_dict = {}
    for case in list(pd.unique(gv_completed_df["Case No"])): #for each case
        temp_df = gv_completed_df[gv_completed_df["Case No"]==case] #filtered by case ONLY
        #counting generic volume by CASE
        if temp_df["Generic Volume"].max() == 0: gv_completed_count = 1 
        else: gv_completed_count = temp_df["Generic Volume"].max()
        #temp_list.append([case, gv_completed_count]) #list of volume by case eg. [[case1,1], [case2,3], [case3,4]]
        temp_dict[case] = gv_completed_count
        
    gv_completed_data.append(temp_dict) #temp_list for each process/team eg. [temp_list of process1teamA, temp_list of process1teamB, etc.]
    gv_completed_list.append(list(pd.unique(gv_completed_df[gv_completed_df["Team Defined SLA (in days)"] < gv_completed_df["Min Time Taken"]]["Case No"])))
    gv_completed_cases.append(list(pd.unique(gv_completed_df["Case No"]))) 
    
    #tests
    #print(process_list[idx], team_list[idx], temp_dict.keys())
    
    #rejected generic volume tasks
    gv_rejected_df = generic_volume_df[(generic_volume_df["Process Type"]==process_list[idx]) &
                                       (generic_volume_df["Currrent Task Status"]=="REJECTED") &
                                       (generic_volume_df["Team"]==team_list[idx])]
    gv_rejected_df = gv_rejected_df[~gv_rejected_df["Case No"].isin(gv_completed_df["Case No"].tolist())] #remove cases already counted under "COMPLETED" so we won't double count

    #sort by case number - this is not for the script, this is for me to visualise better
    gv_rejected_df = gv_rejected_df.sort_values(by=["Case No", "Task Type"])
    
    #count generic volume by case. if 1 case 1 process type multiple task type with gv, take max gv.
    #temp_list = []
    temp_dict = {}
    for case in list(pd.unique(gv_rejected_df["Case No"])): #for each case with same team and same process type
        temp_df = gv_rejected_df[gv_rejected_df["Case No"]==case] #filtered by case ONLY
        #counting generic volume by CASE
        if temp_df["Generic Volume"].max() == 0: gv_rejected_count = 1 
        else: gv_rejected_count = temp_df["Generic Volume"].max()
        #temp_list.append([case, gv_rejected_count]) #list of volume by case eg. [[case1,1], [case2,3], [case3,4]]
        temp_dict[case] = gv_rejected_count
        
    gv_rejected_data.append(temp_dict) #temp_list for each process/team eg. [temp_list of process1teamA, temp_list of process1teamB, etc.]
    gv_rejected_list.append(list(pd.unique(gv_rejected_df[gv_rejected_df["Team Defined SLA (in days)"] < gv_rejected_df["Min Time Taken"]]["Case No"])))
    gv_rejected_cases.append(list(pd.unique(gv_rejected_df["Case No"])))

***

In [ ]:
#case volume: creating remaining dataframe
case_volume_df = team_report_closed_df.drop(generic_volume_df.index, errors='ignore')
case_volume_df = case_volume_df.drop(bulk_creation_df.index, errors='ignore')

In [ ]:
# just mark which cases are past SLA and which arent
case_volume_df['Overdue Days'] = case_volume_df["Team Defined SLA (in days)"] - case_volume_df["Min Time Taken"]

In [ ]:
case_volume_df.to_excel('case_volume_sla.xlsx', index=False)

In [ ]:
#case volume: counting volume
case_completed_data = []
case_completed_list = []
case_completed_cases = []
case_rejected_data = []
case_rejected_list = []
case_rejected_cases = []
for idx in range(len(process_list)):
    #completed cases
    case_completed_df = case_volume_df[(case_volume_df["Process Type"]==process_list[idx]) &
                                       (case_volume_df["Currrent Task Status"]=="COMPLETED") &
                                       (case_volume_df["Team"]==team_list[idx])]
    
    completed_case_list = list(pd.unique(case_completed_df["Case No"]))
    #list compre makes a list of [[case1,1],[case2,1],etc.]. case vol is always 1.
    #case_completed_data.append([[case_list[i],1] for i in range(len(completed_case_list))]) 
    case_completed_data.append({completed_case_list[completed_case_list.index(i)]:1 for i in completed_case_list})
    case_completed_list.append(list(pd.unique(case_completed_df[case_completed_df["Team Defined SLA (in days)"] < case_completed_df["Min Time Taken"]]["Case No"])))
    case_completed_cases.append(list(pd.unique(case_completed_df["Case No"])))
    
    #rejected cases
    case_rejected_df = case_volume_df[(case_volume_df["Process Type"]==process_list[idx]) &
                                      (case_volume_df["Currrent Task Status"]=="REJECTED") &
                                      (case_volume_df["Team"]==team_list[idx])]
    case_rejected_df = case_rejected_df[~case_rejected_df["Case No"].isin(case_completed_df["Case No"].tolist())] #remove cases already counted under "COMPLETED" so we won't double count
    
    rejected_case_list = list(pd.unique(case_rejected_df["Case No"]))
    #list compre makes a list of [[case1,1],[case2,1],etc.]. case vol is always 1.
    #case_rejected_data.append([[case_list[i],1] for i in range(len(rejected_case_list))]) 
    case_rejected_data.append({rejected_case_list[rejected_case_list.index(i)]:1 for i in rejected_case_list})
    case_rejected_list.append(list(pd.unique(case_rejected_df[case_rejected_df["Team Defined SLA (in days)"] < case_rejected_df["Min Time Taken"]]["Case No"])))
    case_rejected_cases.append(list(pd.unique(case_rejected_df["Case No"])))

***

In [ ]:
#total volume for bulk creation, generic volume, case volume - do not sum! take max volume for EACH CASE
#step 1: create set of all the cases for 1 Team/Process combo
#step 2: get all volume values that correspond to each case in the set
#step 3: find max of all the values, sum all the maxs <<-- this is the final volume for 1 process/team

#completed cases
completed_data = []
for i in range(len(bc_completed_data)): #to loop through the list. each list is the same length, bc_completed pick arbitrarily
    case_list = list(bc_completed_data[i].keys()) + list(gv_completed_data[i].keys()) + list(case_completed_data[i].keys())
    case_set = set(case_list)
    x = 0
    for j in case_set:
        templist = [bc_completed_data[i].get(j), gv_completed_data[i].get(j), case_completed_data[i].get(j)]
        temparray = np.array(templist, dtype=np.float64)
        x += np.nanmax(temparray)
    completed_data.append(x)

#rejected cases
rejected_data = []
for i in range(len(bc_rejected_data)): #to loop through the list. each list is the same length, bc_completed pick arbitrarily
    case_list = list(bc_rejected_data[i].keys()) + list(gv_rejected_data[i].keys()) + list(case_rejected_data[i].keys())
    case_set = set(case_list)
    x = 0
    for j in case_set:
        templist = [bc_rejected_data[i].get(j), gv_rejected_data[i].get(j), case_rejected_data[i].get(j)]
        temparray = np.array(templist, dtype=np.float64)
        x += np.nanmax(temparray)
    rejected_data.append(x)

In [ ]:
#completed cases
completed_cases = [", ".join(set(i+j+k)) for i,j,k in zip(bc_completed_cases, gv_completed_cases, case_completed_cases)]
completed_list = [", ".join(set(i+j+k)) for i,j,k in zip(bc_completed_list, gv_completed_list, case_completed_list)]

#rejected cases
rejected_cases = [", ".join(set(i+j+k)) for i,j,k in zip(bc_rejected_cases, gv_rejected_cases, case_rejected_cases)]
rejected_list = [", ".join(set(i+j+k)) for i,j,k in zip(bc_rejected_list, gv_rejected_list, case_rejected_list)]

***

In [ ]:
#median turnaround time
medianTAT_data = []
for idx in range(len(process_list)):
    #data for calculating median per processtype/team
    medianTAT_df = team_report_closed_df[(team_report_closed_df["Process Type"]==process_list[idx]) &
                                         (team_report_closed_df["Currrent Task Status"]!="CANCELLED") &
                                         (team_report_closed_df["Team"]==team_list[idx])]    
    #for case in pd.unique(medianTAT_df["Case No"]):
        #medianTAT_df["Min Time Taken"] = medianTAT_df[medianTAT_df["Case No"]==case]["Min Time Taken"].max()
    #medianTAT_df = medianTAT_df.drop_duplicates(subset="Case No") #take only first instance of each case
    #medianTAT_data.append(medianTAT_df["Min Time Taken"].median()) #finally calculate median
    
    df = medianTAT_df.groupby(by='Case No')[['Min Time Taken']].max().reset_index()
    medianTAT_data.append(df['Min Time Taken'].median())
    
    

***

In [ ]:
data = zip(process_list,team_list,completed_data,rejected_data,medianTAT_data,completed_list,rejected_list,completed_cases,rejected_cases)
output_df = pd.DataFrame(data, columns=["Process Type",
                                        "Team",
                                        "Total Volume (Completed)",
                                        "Total Volume (Rejected)",
                                        "Median TAT (in days)",
                                        "Completed Cases Exceeding SLA",
                                        "Rejected Cases Exceeding SLA",
                                        "All Completed Cases",
                                        "All Rejected Cases"])
output_df = output_df.fillna('NA')

In [ ]:
output_df.to_csv("../Output/CompletedRejectedCases_1.csv", index=False)
print("Completed.")